In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from math import sqrt

In [2]:
pd.__version__
# sklearn.__version__

'2.2.3'

In [3]:
!pip install pyarrow


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [4]:
df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2025-01.parquet')
# df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2025-02.parquet')

In [5]:
df.head

<bound method NDFrame.head of          VendorID tpep_pickup_datetime tpep_dropoff_datetime  passenger_count  \
0               1  2025-01-01 00:18:38   2025-01-01 00:26:59              1.0   
1               1  2025-01-01 00:32:40   2025-01-01 00:35:13              1.0   
2               1  2025-01-01 00:44:04   2025-01-01 00:46:01              1.0   
3               2  2025-01-01 00:14:27   2025-01-01 00:20:01              3.0   
4               2  2025-01-01 00:21:34   2025-01-01 00:25:06              3.0   
...           ...                  ...                   ...              ...   
3475221         2  2025-01-31 23:01:48   2025-01-31 23:16:29              NaN   
3475222         2  2025-01-31 23:50:29   2025-02-01 00:17:27              NaN   
3475223         2  2025-01-31 23:26:59   2025-01-31 23:43:01              NaN   
3475224         2  2025-01-31 23:14:34   2025-01-31 23:34:52              NaN   
3475225         2  2025-01-31 23:56:42   2025-02-01 00:07:27              NaN  

In [6]:
print("Number of columns:", len(df.columns))

Number of columns: 20


In [7]:
# Parse datetime and compute duration in minutes
df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])
df['tpep_dropoff_datetime'] = pd.to_datetime(df['tpep_dropoff_datetime'])
df['duration'] = (df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']).dt.total_seconds() / 60

# STD of duration
print("Standard deviation:", round(df['duration'].std(), 2))

# Filter durations between 1 and 60 minutes
df_filtered = df[(df['duration'] >= 1) & (df['duration'] <= 60)]
fraction = len(df_filtered) / len(df)
print("Fraction remaining:", round(fraction, 2))

Standard deviation: 38.71
Fraction remaining: 0.98


In [8]:
# One-hot encoding using DictVectorizer
df_filtered['PULocationID'] = df_filtered['PULocationID'].astype(str)
df_filtered['DOLocationID'] = df_filtered['DOLocationID'].astype(str)

dicts = df_filtered[['PULocationID', 'DOLocationID']].to_dict(orient='records')

dv = DictVectorizer()
X_train = dv.fit_transform(dicts)
print("Feature matrix shape:", X_train.shape)

/tmp/ipykernel_34613/2820921669.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['PULocationID'] = df_filtered['PULocationID'].astype(str)
/tmp/ipykernel_34613/2820921669.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['DOLocationID'] = df_filtered['DOLocationID'].astype(str)


Feature matrix shape: (3403248, 519)


In [9]:
# Train Linear Regression
y_train = df_filtered['duration'].values
model = LinearRegression()
model.fit(X_train, y_train)

y_pred_train = model.predict(X_train)
rmse_train = sqrt(mean_squared_error(y_train, y_pred_train))
print("Train RMSE:", round(rmse_train, 2))


Train RMSE: 7.71


In [ ]:
# # Load February 2023 data
# df_feb = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2025-02.parquet')

# df_feb['tpep_pickup_datetime'] = pd.to_datetime(df_feb['tpep_pickup_datetime'])
# df_feb['tpep_dropoff_datetime'] = pd.to_datetime(df_feb['tpep_dropoff_datetime'])
# df_feb['duration'] = (df_feb['tpep_dropoff_datetime'] - df_feb['tpep_pickup_datetime']).dt.total_seconds() / 60

# df_feb_filtered = df_feb[(df_feb['duration'] >= 1) & (df_feb['duration'] <= 60)]

# df_feb_filtered['PULocationID'] = df_feb_filtered['PULocationID'].astype(str)
# df_feb_filtered['DOLocationID'] = df_feb_filtered['DOLocationID'].astype(str)

In [ ]:
# dicts_val1 = df_feb_filtered[['PULocationID', 'DOLocationID']].to_dict(orient='records')
# dv1 = DictVectorizer()
# X_val = dv.transform(dicts_val1)
# y_val = df_feb_filtered['duration'].values

# y_pred_val = model.predict(X_val)
# rmse_val = sqrt(mean_squared_error(y_val, y_pred_val, squared=False))
# print("Validation RMSE:", round(rmse_val, 2))